In [21]:

import warnings
warnings.filterwarnings('ignore')

import itertools
import numpy as np
import pandas as pd
from scipy import sparse
from lightfm import LightFM
import matplotlib.pyplot as plt
import seaborn as sns
from lightfm.evaluation import precision_at_k

In [2]:
interaction_train=pd.read_excel("train.xlsx")

In [3]:
interaction_test=pd.read_excel("test.xlsx")

In [4]:
interaction_train.drop("User_ID",axis=1,inplace=True)

In [5]:
interaction_test.drop("User_ID",axis=1,inplace=True)

In [6]:
interaction_train.fillna(0, inplace=True)

In [7]:
interaction_test.fillna(0, inplace=True)

In [8]:
print("Missing values:", interaction_train.isna().sum().sum())

Missing values: 0


In [36]:
interaction_train.shape

(11651, 544)

In [10]:
interaction_test.shape

(11651, 544)

In [31]:
interaction_train=interaction_train[:11651]

In [32]:
interactions_sparse=sparse.csr_matrix(interaction_train.values)

In [33]:
interactions_sparse_test=sparse.csr_matrix(interaction_test.values)

In [34]:
warp_model = LightFM(no_components=15,
                    loss='warp',
                    learning_schedule='adagrad',
                    max_sampled=10,
                    user_alpha=1e-05,
                    item_alpha=1e-05)

In [35]:
warp_model.fit(interactions_sparse)

In [37]:
shape_1 = interaction_train.shape[0]
shape_2 = interaction_train.shape[1]

# to get predictions for all pairs, all combinations should be given as input
combinations = list(zip(*list(itertools.product(range(shape_1),range(shape_2)))))
predictions = warp_model.predict(user_ids=np.array(combinations[0]),
                   item_ids=np.array(combinations[1]))

#rows users, columns items
predictions_df = pd.DataFrame(predictions.reshape(11651, 544))
predictions_df.idxmax(axis=1).value_counts()

99     11270
0        257
3        123
371        1
dtype: int64

In [38]:
print("Train precision: %.2f" % precision_at_k(warp_model, interactions_sparse, k=4).mean())
print("Test precision: %.2f" % precision_at_k(warp_model, interactions_sparse_test, k=4).mean())

Train precision: 0.11
Test precision: 0.10
